In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from patternly.detection import StreamingDetection

data = pd.read_csv("./data/data1", low_memory=False)

In [6]:
pipeline = StreamingDetection(
    window_size=1000,
    window_overlap=0,
    anomaly_sensitivity=2, 
    n_clusters=3, 
    quantize=False, 
    eps=0.2, 
    verbose=True
)
pipeline.fit(data)

predictions = pd.DataFrame(pipeline.predict())
anomalies = predictions[predictions[0] == True]
anomalies
print(list(anomalies.index))

Splitting data into individual streams...
Calculating distance matrix...
Clustering distance matrix...
Generating cluster PFSA 1/3...
Generating cluster PFSA 2/3...
Generating cluster PFSA 3/3...
Calculating cluster PFSA means and stds...
[]


In [3]:
pipeline.save_model()

In [4]:
new_pipeline = StreamingDetection().load_model()

predictions = pd.DataFrame(new_pipeline.predict(data))
anomalies = predictions[predictions[0] == True]
anomalies
print(list(anomalies.index))

Splitting data into individual streams...
[27, 96, 158, 200, 239]


In [7]:
from IPython.display import Image, display
from IPython.core.display import HTML
for i, file in enumerate(pipeline.cluster_PFSA_pngs):
    print(f"Cluster {i} PFSA")
    display(Image(url=f"{file}.png", width=300))

Cluster 0 PFSA


Cluster 1 PFSA


Cluster 2 PFSA
